Transfer learning using cryptobert and roberta

In [1]:
!pip install torch

^C


In [5]:
!pip install protobuf==3.20.0

  Using cached protobuf-3.20.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
google-cloud-bigquery-storage 2.16.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-api-core 2.10.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0

In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 3.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 6.0 MB/s eta 0:00:0000:0100:01


In [6]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig, TextClassificationPipeline
from transformers import pipeline
import numpy as np
import pandas as pd
from scipy.special import softmax
import datetime

2023-04-07 16:36:22.482305: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-07 16:36:23.104625: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-07 16:36:24.884364: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-07 16:36:24.885460: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

## Step 1 - Importing Bitcoin Tweet Data to begin analysing the model

In [8]:
data = pd.read_csv('../data/pre_processed_data/expert_bitcoin_tweets.csv', lineterminator='\n')
# data = data.drop([6690]) #error in row=6690

In [10]:
data.head()

,Unnamed: 0,datetime,username,text\r,preproc_text
0,0,2023-04-03 12:15:20+00:00,saylor,#Bitcoin is Digital Energy. https://t.co/zhAsN...,#Bitcoin is Digital Energy.
1,2,2023-04-02 12:59:03+00:00,saylor,The Robots will only take #Bitcoin. https://t....,The Robots will only take #Bitcoin.
2,3,2023-04-01 21:59:22+00:00,saylor,Gen Z Analyst elegantly articulates why #Bitco...,Gen Z Analyst elegantly articulates why #Bitco...
3,4,2023-04-01 16:45:59+00:00,saylor,#Bitcoin is the Superior Asset. https://t.co/j...,#Bitcoin is the Superior Asset.
4,5,2023-04-01 12:40:19+00:00,saylor,Happy #Bitcoin Fools Day. https://t.co/jSrPtZe...,Happy #Bitcoin Fools Day.


In [11]:
data = data[['datetime','username','text\r']]

In [13]:
data.rename(columns = {'text\r':'text'}, inplace = True)

In [14]:
data

,datetime,username,text
0,2023-04-03 12:15:20+00:00,saylor,#Bitcoin is Digital Energy. https://t.co/zhAsN...
1,2023-04-02 12:59:03+00:00,saylor,The Robots will only take #Bitcoin. https://t....
2,2023-04-01 21:59:22+00:00,saylor,Gen Z Analyst elegantly articulates why #Bitco...
3,2023-04-01 16:45:59+00:00,saylor,#Bitcoin is the Superior Asset. https://t.co/j...
4,2023-04-01 12:40:19+00:00,saylor,Happy #Bitcoin Fools Day. https://t.co/jSrPtZe...
...,...,...,...
27450,2021-04-30 03:56:27+00:00,RektProof,$BTC / $USD\n\n2 Different ranges here LTF/HTF...
27451,2021-04-26 15:03:52+00:00,RektProof,Pretty good start to the year overall.\n\n30R ...
27452,2021-04-24 06:23:55+00:00,RektProof,Power of 3\n\nAnother Round?\n\nOld low broken...
27453,2021-04-19 15:48:27+00:00,RektProof,$BTC / $USD\n\nLooking for a move into 60-62k ...


## Step 2 - Cleaning the data

In [19]:
# I have changed this to remove more information

def preprocess(text):
    new_text = []
    text = str(text)
    text = text.replace("\n", " ")
    for t in text.split(" "):
        t = '' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [20]:
#Create a dummy data
data['process_text'] = data.text
# data['negative_bert'] = data.text
# data['neutral_bert'] = data.text
# data['positive_bert'] = data.text

#use the preprocess to clean the data
data['process_text'] = data['text'].apply(preprocess)

## Step 3 - analyse the language and sentiments by pretrained model

In [15]:
!pyenv local crypto

In [16]:
#Model 2b - Use the bert model, with full scores
MODEL_bert = f"ElKulako/cryptobert"
tokenizer_bert = AutoTokenizer.from_pretrained(MODEL_bert)
tokenizer_bert.model_max_length = 512 #solve the error: RuntimeError: The expanded size of the tensor (562) must match the existing size (514) at non-singleton dimension
config_bert = AutoConfig.from_pretrained(MODEL_bert)


# PT
model_bert = AutoModelForSequenceClassification.from_pretrained(MODEL_bert)
model_bert.config.max_position_embeddings = 512

In [21]:
def scores_bert(sample_text):
    encoded_input_bert = tokenizer_bert(sample_text, return_tensors='pt')
    output_bert = model_bert(**encoded_input_bert)
    scores_bert = output_bert[0][0].detach().numpy()
    scores_bert = softmax(scores_bert) #1st score is negative, 2nd score is netural, 3rd score is positive
    return scores_bert

In [27]:
data['text'] = data['process_text'].apply(scores_bert)

In [31]:
data.to_csv('processed_experts_tweets.csv')

In [29]:
data = data[['date','process_text','text']]


KeyboardInterrupt



In [52]:
data

,date,process_text,text
0,2021-05-01,Good project #punkprotocol #airdrop #giveaway ...,"[0.0003009937, 0.53378063, 0.4659184]"
1,2021-05-01,Demand for Cryptocoin Mixing Rises Significant...,"[0.0013588017, 0.5690318, 0.42960945]"
2,2021-05-01,you are seriously asking me which is my favou...,"[7.4208736e-05, 0.13473623, 0.86518955]"
3,2021-05-01,"We rise again, stronger than ever... #dodgeco...","[0.00022440775, 0.38404274, 0.61573285]"
4,2021-05-01,#Bitcoin this is absolutely irony. Every gover...,"[0.07309254, 0.49250275, 0.4344048]"
...,...,...,...
20014,2021-05-31,"The price of Spaghetti alla Carbonara is 31,87...","[0.023289697, 0.53225714, 0.44445318]"
20015,2021-05-31,"【Our Logo】 Everyone’s dreams, all kinds of dr...","[0.00023141247, 0.24250025, 0.7572683]"
20016,2021-05-31,THE IZOBI ECOSYSTEM OFFERS YOU THE OPTIMAL SOL...,"[0.00013844737, 0.6710398, 0.3288217]"
20017,2021-05-31,Monday morning. Just woke up to win a fucking ...,"[0.0015162149, 0.3077324, 0.69075143]"


In [53]:
data["negative_bert"] = data["text"].map(lambda x: x[0])
data["neutral_bert"] = data["text"].map(lambda x: x[1])
data["positive_bert"] = data["text"].map(lambda x: x[2])

/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_10520/2546257067.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["negative_bert"] = data["text"].map(lambda x: x[0])
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_10520/2546257067.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["neutral_bert"] = data["text"].map(lambda x: x[1])
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_10520/2546257067.py:3: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [54]:
data

,date,process_text,text,negative_bert,neutral_bert,positive_bert
0,2021-05-01,Good project #punkprotocol #airdrop #giveaway ...,"[0.0003009937, 0.53378063, 0.4659184]",0.000301,0.533781,0.465918
1,2021-05-01,Demand for Cryptocoin Mixing Rises Significant...,"[0.0013588017, 0.5690318, 0.42960945]",0.001359,0.569032,0.429609
2,2021-05-01,you are seriously asking me which is my favou...,"[7.4208736e-05, 0.13473623, 0.86518955]",0.000074,0.134736,0.865190
3,2021-05-01,"We rise again, stronger than ever... #dodgeco...","[0.00022440775, 0.38404274, 0.61573285]",0.000224,0.384043,0.615733
4,2021-05-01,#Bitcoin this is absolutely irony. Every gover...,"[0.07309254, 0.49250275, 0.4344048]",0.073093,0.492503,0.434405
...,...,...,...,...,...,...
20014,2021-05-31,"The price of Spaghetti alla Carbonara is 31,87...","[0.023289697, 0.53225714, 0.44445318]",0.023290,0.532257,0.444453
20015,2021-05-31,"【Our Logo】 Everyone’s dreams, all kinds of dr...","[0.00023141247, 0.24250025, 0.7572683]",0.000231,0.242500,0.757268
20016,2021-05-31,THE IZOBI ECOSYSTEM OFFERS YOU THE OPTIMAL SOL...,"[0.00013844737, 0.6710398, 0.3288217]",0.000138,0.671040,0.328822
20017,2021-05-31,Monday morning. Just woke up to win a fucking ...,"[0.0015162149, 0.3077324, 0.69075143]",0.001516,0.307732,0.690751


In [55]:
data = data[['date', 'process_text', 'negative_bert', 'neutral_bert','positive_bert']]

## Step 4: Count the total number of labels/scores (positive, negative vs neutral) by date

In [56]:
grouped_data = pd.DataFrame(data.groupby(['date'])[['negative_bert', 'neutral_bert', 'positive_bert']].sum().fillna(0).reset_index())
grouped_data = grouped_data.rename_axis("", axis="columns")

In [57]:
grouped_data

,date,negative_bert,neutral_bert,positive_bert
0,2021-05-01,25.617947,196.129501,200.252548
1,2021-05-02,21.812752,197.257355,179.929901
2,2021-05-03,31.451441,233.620285,225.928284
3,2021-05-04,22.814571,247.993668,228.191772
4,2021-05-05,33.597691,251.846176,233.556122
5,2021-05-06,24.407339,228.841019,212.751648
6,2021-05-07,29.954519,254.069611,230.975861
7,2021-05-08,25.704597,211.071732,203.223663
8,2021-05-09,18.460251,173.351654,173.188095
9,2021-05-10,31.686495,235.573410,240.740097


## Step 5 - Download the data

In [58]:
start_date = grouped_data['date'][0]
# start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")
start_date

'2021-05-01'

In [59]:
end_date = grouped_data['date'][len(grouped_data)-1]
# end_date_str = datetime.datetime.strftime(end_date, "%Y-%m-%d")
end_date

'2021-05-31'

In [60]:
file_name = f"{start_date}_{end_date}_twitter_comments.csv"
data.to_csv(f"~/code/giadapi/crypto/data/processed/{file_name}")

In [61]:
file_name = f"{start_date}_{end_date}_twitter_transferlearning.csv"
grouped_data.to_csv(f"~/code/giadapi/crypto/data/processed/{file_name}")